#  References

Twitter API with Python: Part 1 -- Streaming Live Tweets, LucidProgramming
- https://www.youtube.com/watch?v=wlnx-7cm4Gg 

Twitter API with Python: Part 2 -- Cursor and Pagination  
- https://www.youtube.com/watch?v=rhBZqEWsZU4&list=PL5tcWHG-UPH2zBfOz40HSzcGUPAVOOnu1

Tweepy
- http://www.tweepy.org/
- http://docs.tweepy.org/en/v3.6.0/
- http://tweepy.readthedocs.io/en/v3.6.0/index.html


In [ ]:
from tweepy import API
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [ ]:
import config #located in same folder

consumer_key = config.twitter_anidata_consumer_key
consumer_secret = config.twitter_anidata_consumer_secret
access_token = config.twitter_anidata_access_token
access_token_secret = config.twitter_anidata_access_token_secret

In [ ]:
# Create a class to print the tweets


class TwitterAuthenticator():
    '''
    Class for authenticating Twitter app
    '''
    
    def authenticate_twitter_app(self):
        auth = OAuthHandler(consumer_key, consumer_secret)        # Create the authentication object
        auth.set_access_token(access_token, access_token_secret)  # Set access token and secret
        return auth      

    
class TwitterStreamer():
    
    """
    Class for streaming and processing live tweets
    """
    
    def __init__(self):
        self.twitter_authenticator = TwitterAuthenticator()
    
    def stream_tweets(self, fetched_tweets_filename, search_terms):
        #This handles twitter authentication and the connection to the twitter streaming API
        
        listener = TwitterListener(fetched_tweets_filename) # Create and object of class TwitterListener
        auth = self.twitter_authenticator.authenticate_twitter_app()
        stream = Stream(auth, listener)
        
        # filter the tweets using filter method provided by stream class
        stream.filter(track=search_terms)
        
        

class TwitterListener(StreamListener): # inherit from TwitterStreamer class which provides methods we can override

    def __init__(self, fetched_tweets_filename): #constructor
        self.fetched_tweets_filename = fetched_tweets_filename
    
    def on_data(self, data):    # take in the data from the StreamListener, then we can do whatever we want with that data
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True             # return True to make sure everything went well
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True  
    
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)
        



In [ ]:
if __name__ == "__main__":
    
    # Define search terms
    search_terms = ['#atlanta', '-filter:retweets']
    
    # Define output file name
    fetched_tweets_filename = "tweets.json"
    
    # Define TwitterStreamer object
    twitter_streamer = TwitterStreamer()
    
    twitter_streamer.stream_tweets(fetched_tweets_filename, search_terms)
    
    